# 금융상품 추천 시스템 개발(CatBoost)

---

## 🎯 프로젝트 개요

### 목적
개인의 특성(나이, 소득, 투자성향 등)을 분석하여 적합한 금융상품을 AI로 추천하는 시스템

### 주요 기능
- **다중 라벨 분류**: 사용자에게 여러 금융상품 동시 추천
- **실제 상품 연동**: 금융감독원 API 데이터와 연결된 실제 상품 정보
- **대화형 입력**: 사용자 친화적인 정보 입력 인터페이스
- **카테고리별 추천**: 예금, 적금, 대출 등 특정 상품군 맞춤 추천

### 사용 기술
- **머신러닝**: RandomForest + MultiOutputClassifier
- **데이터**: 금융감독원 금융상품 통합 비교공시 API
- **평가 지표**: Accuracy, Precision, Recall, F1-Score

---

## 🏗️ 시스템 아키텍처
```
[사용자 입력] → [AI 모델 예측] → [상품 매칭] → [추천 결과]
↓              ↓              ↓            ↓
16개 특성      5개 라벨 확률    JSON 상품DB    맞춤형 추천

### 데이터 흐름
1. **입력**: 사용자의 16개 특성 (나이, 소득, 위험성향 등)
2. **예측**: AI 모델이 5개 금융상품 보유 확률 계산
3. **매칭**: 높은 확률의 카테고리에서 실제 상품 검색
4. **출력**: 금리, 은행명, 가입방법 등 상세 추천


### JSON 매칭 과정 설명

1.  AI 모델이 예측하는 것
AI 모델은 사용자 정보를 보고 **"이 사람이 어떤 금융상품을 보유할 확률"**을 예측합니다.
```
사용자 입력 → AI 모델 → 예측 결과
30세, 5000만원 소득... → 분석 → LIQ: 85%, STOCKS: 20%, CDS: 5%...
```
2.  예측 결과를 상품 카테고리로 변환
AI가 예측한 **라벨(LIQ, STOCKS 등)**을 실제 JSON 파일명으로 연결합니다.
```
pythonLABEL_TO_CATEGORY = {
    'LIQ': 'bank_deposits',      # 유동성자산 → 예금상품 JSON
    'CDS': 'bank_deposits',      # 양도성예금증서 → 예금상품 JSON  
    'NMMF': 'bank_savings',      # 비머니마켓펀드 → 적금상품 JSON
    'STOCKS': 'credit_loans',    # 주식보유 → 신용대출 JSON (투자자금)
    'RETQLIQ': 'bank_savings'    # 퇴직준비금 → 적금상품 JSON
}
예시: AI가 "LIQ 85% 확률"이라고 예측하면 → bank_deposits.json 파일을 열어봅니다.
```
3.  JSON에서 실제 상품 찾기
```
해당 JSON 파일을 열어서 금리가 높은 순서로 실제 상품들을 찾습니다.
bank_deposits.json 열기 → 모든 예금상품 리스트 → 금리 높은 순 정렬 → 상위 3개 선택
```
4.  사용자에게 보여주기
```
AI 예측 + 실제 상품 정보를 합쳐서 사용자에게 추천합니다.
최종 결과:
📂 유동성자산 (85% 확률) ✅ 추천
  1. Sh첫만남우대예금 (수협은행, 2.90%)
  2. NH고향사랑기부예금 (농협은행, 2.80%)
```

### 전체 흐름 요약
```
사용자 입력 → 나이, 소득 등
AI 예측 → LIQ 85%, STOCKS 20% 등
매핑 테이블 → LIQ = bank_deposits.json
JSON 검색 → 예금상품들 중 금리 높은 것들
최종 추천 → 구체적인 은행 상품명과 금리

핵심은 AI가 "어떤 종류의 상품이 적합한지" 예측하면, 그 종류에 해당하는 실제 상품들을 JSON에서 찾아서 보여주는 것입니다! 🎯
```


## 💻 코드 구조 및 설명

### 1. 환경 설정 및 라이브러리
**설명**: 데이터 처리, 머신러닝 모델, 성능 평가에 필요한 라이브러리들을 불러옵니다.

In [12]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
warnings.filterwarnings('ignore')

In [13]:
print("CatBoost 모델 훈련 시작...")

#데이터 로드
df = pd.read_csv('../2.전처리/data/SCFP/cleaned_scf_data.csv')
X = df.iloc[:,:-5]  # 14개 독립변수
y = df.iloc[:,-5:]  # 5개 타겟변수

print(f"데이터 크기: {X.shape[0]:,}명, 특성: {X.shape[1]}개, 타겟: {y.shape[1]}개")

CatBoost 모델 훈련 시작...
데이터 크기: 22,975명, 특성: 14개, 타겟: 5개


In [14]:
# 데이터 분할
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:


# 다중라벨 CatBoost 모델
base_model = CatBoostClassifier(
    iterations=500,
    depth=16,
    learning_rate=0.1,                # 학습률
    l2_leaf_reg=10,
    auto_class_weights='Balanced',
    # 범주형 특성 설정 ★★★ 중요 ★★★
    cat_features=categorical_features,
    random_seed=42,
    thread_count=-1,              # 병렬 처리
)

multilabel_model = MultiOutputClassifier(base_model)
multilabel_model.fit(train_X, train_y)

# 모델 성능 평가
test_pred = multilabel_model.predict(test_X)

# 전체 성능 지표
accuracy = accuracy_score(test_y, test_pred)
precision = precision_score(test_y, test_pred, average='samples')
recall = recall_score(test_y, test_pred, average='samples')
f1 = f1_score(test_y, test_pred, average='samples')

print("★ CatBoost 모델 훈련 완료!")
print("\n★ 전체 모델 성능")
print("=" * 40)
print(f"정확도 (Accuracy):  {accuracy:.3f}")
print(f"정밀도 (Precision): {precision:.3f}")
print(f"재현율 (Recall):    {recall:.3f}")
print(f"F1 점수:           {f1:.3f}")

# 개별 상품별 성능
print("\n★ 상품별 성능 지표")
print("=" * 40)

target_names_list = ['유동성자산', '양도성예금증서', '비머니마켓펀드', '주식보유', '퇴직준비금유동성']

for i, name in enumerate(target_names_list):
    y_true_i = test_y.iloc[:, i]
    y_pred_i = test_pred[:, i]
    
    acc_i = accuracy_score(y_true_i, y_pred_i)
    prec_i = precision_score(y_true_i, y_pred_i, zero_division=0)
    rec_i = recall_score(y_true_i, y_pred_i, zero_division=0)
    f1_i = f1_score(y_true_i, y_pred_i, zero_division=0)
    
    print(f"\n● {name}")
    print(f"   정확도: {acc_i:.3f} | 정밀도: {prec_i:.3f} | 재현율: {rec_i:.3f} | F1: {f1_i:.3f}")

# 성능 해석
print("\n★ 성능 해석")
print("=" * 40)
if accuracy >= 0.7:
    print("★ 우수한 전체 정확도!")
elif accuracy >= 0.6:
    print("◎ 양호한 전체 정확도")
else:
    print("※ 정확도 개선 필요")

if precision >= 0.8:
    print("★ 높은 정밀도 - 추천 신뢰성 우수!")
elif precision >= 0.7:
    print("◎ 양호한 정밀도")
else:
    print("※ 정밀도 개선 필요 - 잘못된 추천 많음")

if recall >= 0.7:
    print("★ 높은 재현율 - 놓치는 상품 적음!")
elif recall >= 0.6:
    print("◎ 양호한 재현율")
else:
    print("※ 재현율 개선 필요 - 놓치는 상품 많음")

0:	learn: 0.6425704	total: 203ms	remaining: 1m 41s
1:	learn: 0.5439927	total: 426ms	remaining: 1m 45s
2:	learn: 0.5140729	total: 466ms	remaining: 1m 17s
3:	learn: 0.4931248	total: 508ms	remaining: 1m 2s
4:	learn: 0.4447547	total: 680ms	remaining: 1m 7s
5:	learn: 0.4015695	total: 798ms	remaining: 1m 5s
6:	learn: 0.3477391	total: 1.6s	remaining: 1m 52s
7:	learn: 0.3316701	total: 1.68s	remaining: 1m 43s
8:	learn: 0.2932399	total: 3.12s	remaining: 2m 50s
9:	learn: 0.2823291	total: 3.17s	remaining: 2m 35s
10:	learn: 0.2735175	total: 3.26s	remaining: 2m 24s
11:	learn: 0.2607833	total: 3.52s	remaining: 2m 23s
12:	learn: 0.2339830	total: 5.71s	remaining: 3m 34s
13:	learn: 0.2149549	total: 11.9s	remaining: 6m 52s
14:	learn: 0.2142461	total: 11.9s	remaining: 6m 25s
15:	learn: 0.2127603	total: 11.9s	remaining: 6m 1s
16:	learn: 0.2085469	total: 12s	remaining: 5m 41s
17:	learn: 0.1919960	total: 18.4s	remaining: 8m 11s
